# Assignment 02

- Name: Ran Arino
- Student ID: 153073200
- Email: rarino@myseneca.ca
- Course: Machine Learning
- Course ID: BDA500NAA.05380.2237
- Professor: Dr. Amir Moslemi

In [27]:
import numpy as np
import pandas as pd

import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow.python'

In [2]:
# load dataset
from mat4py import loadmat 
data = loadmat('CatDog.mat')

In [18]:
data['ny']

64

In [11]:
data.keys()

dict_keys(['X', 'G', 'nx', 'ny'])

In [25]:
type(data['X'])

list